In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==========================================
# 0. Google Drive 마운트 (모델 경로가 드라이브에 있으므로 필수)
# ==========================================
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    print("📂 Google Drive를 마운트합니다...")
    drive.mount('/content/drive')
else:
    print("✅ Google Drive가 이미 마운트되어 있습니다.")

# ==========================================
# 1. 라이브러리 설치
# ==========================================
!pip install -qU fastapi uvicorn pyngrok nest_asyncio transformers accelerate bitsandbytes

import torch
import uvicorn
import nest_asyncio
import threading  # [중요] 스레딩 모듈 추가
from fastapi import FastAPI, Request
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 2. Ngrok 설정
# ==========================================
# 입력해주신 토큰을 적용했습니다.
NGROK_AUTH_TOKEN = "36ByM2ypIJcCEm3rImpa733CybJ_6eou5fv1gEgqYfbMfFN5L"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ==========================================
# 3. 모델 로드 (Qwen2-Instruct + 파인튜닝 어댑터)
# ==========================================
print("⏳ 모델을 로드 중입니다... (시간이 조금 걸립니다)")

# 사용자의 구글 드라이브 경로
model_path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.4"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",          # GPU 자동 할당
        torch_dtype=torch.float16,  # 메모리 절약
        load_in_4bit=True,        # VRAM 부족 시 주석 해제
        trust_remote_code=True
    )
    print("✅ 모델 로드 완료!")
except OSError as e:
    print(f"❌ 모델 로드 실패! 경로를 확인해주세요: {model_path}")
    print(f"에러 메시지: {e}")
    # 경로가 틀렸을 경우를 대비해 중단
    raise e

# ==========================================
# 4. FastAPI 서버 정의
# ==========================================
app = FastAPI()

@app.post("/generate")
async def generate(request: Request):
    data = await request.json()

    # 로컬(LangGraph)에서 온 프롬프트 받기
    prompt = data.get("prompt", "")

    # LangGraph가 지시문(System Prompt)을 포함해서 보내므로
    # 여기서는 'user' 메시지로 래핑하여 모델에게 전달합니다.
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Chat Template 적용
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 텍스트 생성
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=0.1,  # 낮은 온도로 정확성 높임
        do_sample=True
    )

    # 결과 디코딩
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return {"response": response_text}

# ==========================================
# 5. 서버 실행 (백그라운드 스레드 방식)
# ==========================================
# 기존 터널 종료
ngrok.kill()

# 8000번 포트 노출
tun = ngrok.connect(8000)

print(f"\n🎉 서버가 백그라운드에서 시작되었습니다!")
print(f"👉 Public URL: {tun.public_url}")
print(f"\n위 URL을 복사해서 로컬 PC의 .env 파일에 붙여넣으세요.")

# [핵심 수정] Uvicorn을 별도 스레드에서 실행하여 Colab 멈춤 방지
def run_server():
    uvicorn.run(app, port=8000)

server_thread = threading.Thread(target=run_server)
server_thread.start()

📂 Google Drive를 마운트합니다...
Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires fastapi<0.119.0,>=0.115.0, but you have fastapi 0.123.0 which is incompatible.
⏳ 모델을 로드 중입니다... (시간이 조금 걸립니다)


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Qwen2로 BHC, DI생성/ Qwen3로 번역

In [ ]:
# ==========================================
# 0. Google Drive 마운트 (Qwen2 파인튜닝 모델 경로)
# ==========================================
from google.colab import drive
import os
import re   # <think> 제거용
import json # route_action JSON 파싱용

if not os.path.exists('/content/drive'):
    print("📂 Google Drive를 마운트합니다...")
    drive.mount('/content/drive')
else:
    print("✅ Google Drive가 이미 마운트되어 있습니다.")

# ==========================================
# 1. 라이브러리 설치
# ==========================================
# FastAPI: 웹서버 프레임워크
# uvicorn: 서버 실행기
# pyngrok: 외부 접속용 터널링 도구
# bitsandbytes: 모델 양자화(메모리 절약용)
!pip install -qU fastapi uvicorn pyngrok nest_asyncio "transformers>=4.51.0" accelerate bitsandbytes

import torch
import uvicorn
import nest_asyncio
import threading
from fastapi import FastAPI, Request
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

# Colab의 이벤트 루프와 uvicorn이 충돌하지 않게 패치
# Colab은 이미 비동기 루프가 돌아가고 있는데 여기에 FastAPI서버를 또 돌리면 충돌이난다. 이를 해결해주기 위한 코드이다.
nest_asyncio.apply()

# ==========================================
# 2. Ngrok 설정 -> 서버를 인터넷에서 접속할 수 있는 공인 URL로 만들어 준다.
# ==========================================
NGROK_AUTH_TOKEN = "36ByM2ypIJcCEm3rImpa733CybJ_6eou5fv1gEgqYfbMfFN5L"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ==========================================
# 3. 모델 로드
#    - Qwen2 (파인튜닝된 BHC/DI 생성용)
#    - Qwen3-8B (번역 + 에이전트용)
# ==========================================
print("⏳ Qwen2 파인튜닝 모델을 로드 중입니다...")

q2_model_path = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.4"

try:
    q2_tokenizer = AutoTokenizer.from_pretrained(q2_model_path, trust_remote_code=True)
    q2_model = AutoModelForCausalLM.from_pretrained(
        q2_model_path,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
        # 필요하면 load_in_4bit=True 로 변경 가능
    )
    print("✅ Qwen2 로드 완료!")
except OSError as e:
    print(f"❌ Qwen2 모델 로드 실패! 경로를 확인해주세요: {q2_model_path}")
    print(f"에러 메시지: {e}")
    raise e

print("⏳ Qwen3-8B 번역/에이전트용 모델을 로드 중입니다...")

# Hugging Face Hub ID로 Qwne3-8B라는 이름을 가진 공식 Qwen3모델의 주소임.
# 한 번 다운로드하면 캐시에 저장
q3_model_name = "Qwen/Qwen3-8B"

try:
    q3_tokenizer = AutoTokenizer.from_pretrained(q3_model_name)
    q3_model = AutoModelForCausalLM.from_pretrained(
        q3_model_name,
        device_map="auto",
        torch_dtype="auto",
        load_in_4bit=True,   # VRAM 절약
    )
    print("✅ Qwen3-8B 로드 완료!")
except Exception as e:
    print("❌ Qwen3-8B 로드 중 오류 발생:", e)
    raise e

# ==========================================
# 4. 번역용 시스템 프롬프트 (Qwen3용)
# ==========================================
TRANSLATE_PROMPT_KO = """
You are a professional medical translator.

TASK
- Translate the following hospital discharge summary from English to Korean.

REQUIREMENTS
- Keep all clinical information and medical facts exactly the same. Do NOT add or remove facts.
- The source text may contain TWO main sections:
  - "Brief Hospital Course"
  - "Discharge Instructions"
- You MUST translate ALL sections from the source text. Do NOT omit any part.
- Translate headings:
  - "Brief Hospital Course" → "입원 경과 요약"
  - "Discharge Instructions" → "퇴원 지침"
- Preserve the overall structure (headings, bullet points, line breaks).
- Keep medication names, doses, units, lab values, and times exactly as written (do NOT translate drug names).
- Use formal and polite Korean suitable for a written medical document.
- Do NOT output your thoughts, analysis, or any <think> tags.
- Return ONLY the final Korean translation text, with no explanations, comments, or extra sentences.
"""

# ==========================================
# 4-2. 에이전트(행동 결정)용 시스템 프롬프트 (Qwen3)
# ==========================================
AGENT_SYSTEM_PROMPT = """
You are an assistant that decides what action the application should take
based on the user's Korean command and the current discharge summary.

You MUST respond with STRICT JSON ONLY. No explanation, no extra text.

Possible actions:
- "save_file": user wants to create a file (PDF or DOCX) from the current summary.
- "send_email": user wants to send the current summary as a file by email.
- "none": user is just asking a question or something that does not require these actions.

Rules:
- "file_type" must be either "pdf" or "docx" (default to "docx" if not clear).
- If the user mentions an email address (like aaa@bbb.com), put it in "email".
- If there is no email address and the action is "send_email", set "email" to "".

Output JSON format (no other fields, all lowercase keys):

{
  "action": "save_file" | "send_email" | "none",
  "file_type": "pdf" | "docx",
  "email": "string"
}
"""

# ==========================================
# 5. 헬퍼 함수
# ==========================================
def run_qwen2_generate(raw_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    Qwen2 파인튜닝 모델로 영어 BHC/DI 생성
    - raw_prompt: (중요) 로컬에서 이미 system 지시문 + 케이스 텍스트를 합쳐서 보내는 문자열
                  예전 /generate 코드와 동일하게, 그냥 user 메시지 하나로 처리
    """
    messages = [
        {"role": "user", "content": raw_prompt}
    ]

    text = q2_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = q2_tokenizer([text], return_tensors="pt").to(q2_model.device)

    generated_ids = q2_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.1,
        do_sample=True,
    )

    trimmed_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response_text = q2_tokenizer.batch_decode(trimmed_ids, skip_special_tokens=True)[0]
    return response_text.strip()


def run_qwen3_translate_to_ko(english_text: str, max_new_tokens: int = 1024) -> str:
    """
    Qwen3-8B로 영어 → 한국어 번역
    - <think>...</think> 블록 제거
    - BHC + DI 전체를 그대로 번역 (프롬프트에서 강제)
    """
    messages = [
        {"role": "system", "content": TRANSLATE_PROMPT_KO},
        {"role": "user", "content": english_text},
    ]
    text = q3_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    model_inputs = q3_tokenizer([text], return_tensors="pt").to(q3_model.device)

    generated_ids = q3_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.2,
        do_sample=False,
    )

    trimmed_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    ko_text = q3_tokenizer.batch_decode(trimmed_ids, skip_special_tokens=True)[0]

    # 혹시라도 <think>...</think> 가 들어오면 전부 제거
    ko_text = re.sub(r"<think>.*?</think>", "", ko_text, flags=re.DOTALL)

    return ko_text.strip()


def run_qwen3_route_action(command: str, summary_ko: str, max_new_tokens: int = 256) -> dict:
    """
    Qwen3-8B로 사용자 명령(한국어)을 해석해서
    - action: "save_file" | "send_email" | "none"
    - file_type: "pdf" | "docx"
    - email: "..."
    형태의 JSON 딕셔너리를 반환
    """
    messages = [
        {"role": "system", "content": AGENT_SYSTEM_PROMPT},
        {
            "role": "user",
            "content": f"User command (Korean): {command}\n\nCurrent summary (Korean):\n{summary_ko}",
        },
    ]
    text = q3_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,  # <think> 방지
    )
    model_inputs = q3_tokenizer([text], return_tensors="pt").to(q3_model.device)

    generated_ids = q3_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.0,
        do_sample=False,
    )

    trimmed_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    raw = q3_tokenizer.batch_decode(trimmed_ids, skip_special_tokens=True)[0].strip()

    # ```json ... ``` 처럼 감싸져 올 수 있으니 안쪽 JSON만 추출
    raw_clean = raw.strip()
    if raw_clean.startswith("```"):
        # ```json\n{...}\n``` 이런 형태 대비
        raw_clean = raw_clean.strip("`")
        idx = raw_clean.find("{")
        if idx != -1:
            raw_clean = raw_clean[idx:]
    else:
        raw_clean = raw_clean

    try:
        data = json.loads(raw_clean)
    except Exception as e:
        print("[run_qwen3_route_action] JSON 파싱 실패, raw:", raw_clean[:200])
        print("에러:", e)
        data = {"action": "none", "file_type": "docx", "email": ""}

    # 기본값 보정
    if data.get("action") not in ["save_file", "send_email", "none"]:
        data["action"] = "none"
    if data.get("file_type") not in ["pdf", "docx"]:
        data["file_type"] = "docx"
    if "email" not in data:
        data["email"] = ""

    return data

# ==========================================
# 6. FastAPI 서버 정의
# ==========================================
app = FastAPI()

# (1) Qwen2로 영어 BHC/DI 생성만 하는 엔드포인트 -> 사용은 안 하지만 디버깅, 확장용 API
@app.post("/generate_en")
async def generate_en(request: Request):
    data = await request.json()
    prompt = data.get("prompt", "")
    if not prompt:
        return {"error": "prompt is required"}

    print("[/generate_en] 요청 수신, prompt 길이:", len(prompt))
    response_en = run_qwen2_generate(prompt)
    print("[/generate_en] 생성 완료, 영어 길이:", len(response_en))
    return {"response_en": response_en}


# (2) Qwen3로 영어 텍스트를 한국어로 번역하는 엔드포인트 -> 사용은 안 하지만 디버깅, 확장용 API
@app.post("/translate_ko")
async def translate_ko(request: Request):
    data = await request.json()
    english_text = data.get("text", "")
    if not english_text:
        return {"error": "text (english) is required"}

    print("[/translate_ko] 요청 수신, 영어 길이:", len(english_text))
    response_ko = run_qwen3_translate_to_ko(english_text)
    print("[/translate_ko] 번역 완료, 한국어 길이:", len(response_ko))
    return {"response_ko": response_ko}


# (3) 한 번에 영어 생성 + 한국어 번역까지 해서 돌려주는 엔드포인트
@app.post("/generate_ko")
async def generate_ko(request: Request):
    """
    - 입력: {"prompt": "...(BHC/DI 지시문 + 입원 전체 정보)..." }
      👉 여기 prompt에는 예전처럼 이미 "You are an expert physician..." 같은 지시문을
         로컬 쪽에서 붙여서 보내는 게 좋습니다.
    - Qwen2: 영어 BHC/DI 생성
    - Qwen3: 그 영어 BHC/DI를 한국어로 번역
    - 출력: {"response_en": "...", "response_ko": "..."}
    """
    data = await request.json()
    prompt = data.get("prompt", "")
    if not prompt:
        return {"error": "prompt is required"}

    print("[/generate_ko] 요청 수신, prompt 길이:", len(prompt))

    # 1단계: Qwen2로 영어 BHC/DI 생성
    print("[/generate_ko] Qwen2 생성 시작")
    response_en = run_qwen2_generate(prompt)
    print("[/generate_ko] Qwen2 생성 완료, 영어 길이:", len(response_en))

    # 2단계: Qwen3로 한국어 번역
    print("[/generate_ko] Qwen3 번역 시작")
    response_ko = run_qwen3_translate_to_ko(response_en)
    print("[/generate_ko] Qwen3 번역 완료, 한국어 길이:", len(response_ko))

    return {
        "response_en": response_en,
        "response_ko": response_ko,
    }


# (4) 에이전트 액션 라우팅 엔드포인트 (Qwen3 사용)
@app.post("/route_action")
async def route_action(request: Request):
    """
    - 입력: {"command": "...", "summary_ko": "..."}
    - 출력: {"action": "...", "file_type": "...", "email": "..."}
    """
    data = await request.json()
    command = data.get("command", "")
    summary_ko = data.get("summary_ko", "")

    if not command:
        return {"error": "command is required"}

    print("[/route_action] 요청 수신:", command[:80], "...")
    result = run_qwen3_route_action(command, summary_ko)
    print("[/route_action] 결정:", result)
    return result

# ==========================================
# 6-b. Qwen3-8B 에이전트용 채팅 엔드포인트
#      (LangGraph 에이전트에서 사용)
# ==========================================
@app.post("/agent_chat")
async def agent_chat(request: Request):
    """
    LangGraph 에이전트가 사용하는 Qwen3-8B용 단일 채팅 엔드포인트.
    입력: {"prompt": "..."}  (system + user까지 모두 합친 문자열)
    출력: {"response": "<모델 출력 텍스트>"}
    """
    data = await request.json()
    prompt = data.get("prompt", "")

    if not prompt:
        return {"response": ""}

    messages = [
        {"role": "user", "content": prompt}
    ]

    text = q3_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        # enable_thinking=False  # Qwen3 계열에서 이 옵션 쓰는 버전이면 끄기
    )

    model_inputs = q3_tokenizer([text], return_tensors="pt").to(q3_model.device)

    generated_ids = q3_model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.3,
        do_sample=True,
        top_p=0.9,
    )

    trimmed_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response_text = q3_tokenizer.batch_decode(trimmed_ids, skip_special_tokens=True)[0]

    return {"response": response_text.strip()}


# ==========================================
# 7. 서버 실행 (백그라운드 스레드)
# ==========================================
ngrok.kill()
tun = ngrok.connect(8000)

print(f"\n🎉 서버가 백그라운드에서 시작되었습니다!")
print(f"👉 Public URL: {tun.public_url}")
print(f"    - 영어만 생성: POST {tun.public_url}/generate_en")
print(f"    - 영어→한국어 번역: POST {tun.public_url}/translate_ko")
print(f"    - 한 번에 생성+번역: POST {tun.public_url}/generate_ko")
print(f"    - 액션 라우팅: POST {tun.public_url}/route_action\n")

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=run_server)
server_thread.start()


📂 Google Drive를 마운트합니다...
Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 150.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires fastapi<0.119.0,>=0.115.0, but you have fastapi 0.123.0 which is incompatible.
⏳ Qwen2 파인튜닝 모델을 로드 중입니다...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Qwen2 로드 완료!
⏳ Qwen3-8B 번역/에이전트용 모델을 로드 중입니다...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Qwen3-8B 로드 완료!

🎉 서버가 백그라운드에서 시작되었습니다!
👉 Public URL: https://nondedicatory-ruth-silvicolous.ngrok-free.dev
    - 영어만 생성: POST https://nondedicatory-ruth-silvicolous.ngrok-free.dev/generate_en
    - 영어→한국어 번역: POST https://nondedicatory-ruth-silvicolous.ngrok-free.dev/translate_ko
    - 한 번에 생성+번역: POST https://nondedicatory-ruth-silvicolous.ngrok-free.dev/generate_ko
    - 액션 라우팅: POST https://nondedicatory-ruth-silvicolous.ngrok-free.dev/route_action

